<a href="https://colab.research.google.com/github/Ronbragaglia/ColabPython/blob/main/Atividade1_PERCEPTRON_AND_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pathlib
import pickle
import random

from sklearn.neural_network import MLPClassifier
from pathlib import Path

# Perceptron

Foram criadas as funções para treino e utilização do perceptron. O treino é feito com o dataset fornecido para a classificação de Laranja e Tangerina.

O teste é realizado com os seguintes dados:
```python
X_test = [(0.09, 0.35, 0.7),
          (0.5, 0.8, 0.7),
          (0.35, 0.78, 0.29)]

y_test = [1, 0, 0]
```

Estes valores não estão presentes nos dados de treino e foram criados alterando entre 0.01 e 0.1 dos atributos de tangerina e laranja.

In [3]:
def perceptron_train(bias: float, learning_rate: float, epochs: int, train_data: list[tuple]) -> Path:
    """Função para treinar um single perceptron.

    :param bias: x0
    :param learning_rate: taxa de aprendizagem
    :param epochs: Limite de épocas que devem ser rodadas
    :param train_data: dados que serão utilizados para treinar o perceptron
    [(<VALOR_1>, <VALOR_2>, <CLASSIFICAÇÃO>), ..., (<VALOR_1>, <VALOR_2>, <CLASSIFICAÇÃO>)]
    :return: pickle file contendo bias, w0, w1, w2 como os novos pesos.
    """

    start_weights = [round(random.uniform(1, -1), 1) for _ in range(len(train_data[0]))]

    for epoch in range(1, epochs+1):
        print(f"{'='*20} [EPOCH {epoch:02d}/{epochs:02d}] {'='*20}")

        ABSOLUTE_ERROR = 0

        for idx, data in enumerate(train_data):
            print(f"{'=-'*10} [SAMPLE {idx+1:02d}/{len(train_data):02d}] {'=-'*10}")


            d = data[-1]

            attributes = data[0:-1]

            S = bias*start_weights[0]

            for attrXi, weightsWi in zip(attributes, start_weights[1:]):
                S += attrXi * weightsWi

            print(f"{S=}")

            Y = None

            if S >= 0:
                Y = 1
            elif S < 0:
                Y = 0

            LOSS = d - Y
            print(f"{Y=}\n{LOSS=}")

            ABSOLUTE_ERROR += abs(LOSS)

            if LOSS != 0:
                start_weights[0] = start_weights[0]+learning_rate*(d-Y)*bias

                for idx, weights in enumerate(zip(attributes, start_weights[1:])):
                    attr, weight = weights
                    start_weights[idx+1] = weight+learning_rate*(d-Y)*attr

                print(f"Pesos alterados! {start_weights}")

        if ABSOLUTE_ERROR == 0:
            print(f"RNA Treinada! Os pesos são: {start_weights}")
            break

    weights = [bias] + start_weights
    weights_path_name = pathlib.Path(r'weights')
    weights_path_name.mkdir(exist_ok=True)

    with open(weights_path_name / 'weights.pkl', 'wb') as f:
        f.write(pickle.dumps(weights))

    print("Modelo salvo em", weights_path_name)
    return weights_path_name / 'weights.pkl'


def perceptron_classifier(weights_pkl: Path, *args):
    """Função para utilizar o perceptron treinado.

    :return: 0 || 1.
    """

    with open(weights_pkl, 'rb') as f:
        weights: list = pickle.load(f)

    S = weights.pop(0)*weights.pop(0)

    for Xi, Wi in zip(args, weights):
        print(f'{Xi=} e {Wi=}')
        S += Xi*Wi

    # bias, w0, w1, w2 = weights
    #
    # S = (bias*w0)+(a*w1)+(b*w2)

    Y = None

    if S >= 0:
        Y = 1
    elif S < 0:
        Y = 0

    return Y

In [4]:
train_data = [(0.1, 0.4, 0.7, 1.),
              (0.5, 0.7, 0.1, 1.),
              (0.6, 0.9, 0.8, 0.),
              (0.3, 0.7, 0.2, 0.)]

weights_path = perceptron_train(bias=1.0,
                                learning_rate=0.5,
                                epochs=100,
                                train_data=train_data)

X_test = [(0.09, 0.35, 0.7),
          (0.5, 0.8, 0.7),
          (0.35, 0.78, 0.29)]

y_test = [1, 0, 0]

for idx, data in enumerate(X_test):
  fosforo, acidez, calcio = data

  classifica = perceptron_classifier(weights_path, fosforo, acidez, calcio)

  classifica_desc = 'Tangerina' if classifica else 'Laranja'

  print(f"Para {fosforo=}, {acidez=}, {calcio=} o perceptron deu a classificação de {classifica=} ({classifica_desc}). O valor correto para esses dados é {y_test[idx]}.")


==================== [EPOCH 01/100] ====================
=-=-=-=-=-=-=-=-=-=- [SAMPLE 01/04] =-=-=-=-=-=-=-=-=-=-
S=1.3599999999999999
Y=1
LOSS=0.0
=-=-=-=-=-=-=-=-=-=- [SAMPLE 02/04] =-=-=-=-=-=-=-=-=-=-
S=1.2100000000000002
Y=1
LOSS=0.0
=-=-=-=-=-=-=-=-=-=- [SAMPLE 03/04] =-=-=-=-=-=-=-=-=-=-
S=1.9500000000000002
Y=1
LOSS=-1.0
Pesos alterados! [0.09999999999999998, 0.6000000000000001, -0.35, 0.5]
=-=-=-=-=-=-=-=-=-=- [SAMPLE 04/04] =-=-=-=-=-=-=-=-=-=-
S=0.13500000000000006
Y=1
LOSS=-1.0
Pesos alterados! [-0.4, 0.45000000000000007, -0.7, 0.4]
==================== [EPOCH 02/100] ====================
=-=-=-=-=-=-=-=-=-=- [SAMPLE 01/04] =-=-=-=-=-=-=-=-=-=-
S=-0.35500000000000004
Y=0
LOSS=1.0
Pesos alterados! [0.09999999999999998, 0.5000000000000001, -0.49999999999999994, 0.75]
=-=-=-=-=-=-=-=-=-=- [SAMPLE 02/04] =-=-=-=-=-=-=-=-=-=-
S=0.07500000000000012
Y=1
LOSS=0.0
=-=-=-=-=-=-=-=-=-=- [SAMPLE 03/04] =-=-=-=-=-=-=-=-=-=-
S=0.5500000000000002
Y=1
LOSS=-1.0
Pesos alterados! [-0.4, 0.20

# W/ MLP

Foi realizado o treinamento do modelo utilizando o MLPRegressor da biblioteca scikit-learn utilizando 1 hidden layer com 4 neurons e o número máximo de épocas limitado em 4000. Para o teste foi utilizado os mesmos dados de testes criados para testar o modelo utilizando o single perceptron. O resultado das classificações foram iguais aos do Perceptron como esperado.

In [ ]:
# Dados de treinamento
X_treino = [
    [0.1, 0.4, 0.7],
    [0.5, 0.7, 0.1],
    [0.6, 0.9, 0.8],
    [0.3, 0.7, 0.2]
]

y_treino = [1, 1, -1, -1]

# Criação e treinamento de uma MLP
mlp = MLPClassifier(hidden_layer_sizes=(4, ), max_iter=4000)
mlp.fit(X_treino, y_treino)

# Exemplos "parecidos"
exemplo1 = [0.09, 0.35, 0.7]
exemplo2 = [0.5, 0.8, 0.7]
exemplo3 = [0.35, 0.78, 0.29]

# Teste dos modelos com os exemplos "parecidos"
previsao_mlp = mlp.predict([exemplo1, exemplo2, exemplo3])

print("\nResultados do teste com MLP:")
for i, previsao in enumerate(previsao_mlp):
    classe = "tangerina" if previsao == 1 else "laranja"
    print(f"Exemplo {i + 1}: Classe prevista: {classe}")


Resultados do teste com MLP:
Exemplo 1: Classe prevista: tangerina
Exemplo 2: Classe prevista: laranja
Exemplo 3: Classe prevista: laranja
